In [58]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import numpy as np
from collections import Counter
from tqdm import tqdm
import torchvision.models as models
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
import torch.optim as optim
from torchvision.models import convnext_tiny
from sklearn.model_selection import train_test_split
import timm

In [59]:
SEED = 42 
np.random.seed(SEED) 
torch.manual_seed(SEED)

In [60]:
class DERMNET_DATASET(Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        super().__init__()
        self.root_dir = root_dir
        self.train = train
        self.transform = transform

        self.labels_dict = {
            'Light Diseases and Disorders of Pigmentation': 0, 
            'Lupus and other Connective Tissue diseases': 1,
            'Acne and Rosacea Photos': 2,
            'Systemic Disease': 3,
            'Poison Ivy Photos and other Contact Dermatitis': 4,
            'Vascular Tumors': 5,
            'Urticaria Hives': 6,
            'Atopic Dermatitis Photos': 7,
            'Bullous Disease Photos': 8,
            'Hair Loss Photos Alopecia and other Hair Diseases': 9,
            'Tinea Ringworm Candidiasis and other Fungal Infections': 10,
            'Psoriasis pictures Lichen Planus and related diseases': 11,
            'Melanoma Skin Cancer Nevi and Moles': 12,
            'Nail Fungus and other Nail Disease': 13,
            'Scabies Lyme Disease and other Infestations and Bites': 14,
            'Eczema Photos': 15,
            'Exanthems and Drug Eruptions': 16,
            'Herpes HPV and other STDs Photos': 17,
            'Seborrheic Keratoses and other Benign Tumors': 18,
            'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions': 19,
            'Vasculitis Photos': 20,
            'Cellulitis Impetigo and other Bacterial Infections': 21,
            'Warts Molluscum and other Viral Infections': 22
        }

        dataset_type = 'train' if self.train else 'test'
        dataset_dir = os.path.join(root_dir, dataset_type)

        # Tạo một dictionary để lưu danh sách ảnh cho từng nhãn
        label_to_images = {label: [] for label in range(len(self.labels_dict))}

        # Duyệt qua các thư mục con và lưu đường dẫn ảnh theo nhãn tương ứng
        for class_name, class_idx in self.labels_dict.items():
            class_folder = os.path.join(dataset_dir, class_name)
            if os.path.isdir(class_folder):
                image_paths = sorted([os.path.join(class_folder, img) 
                                      for img in os.listdir(class_folder) 
                                      if img.endswith(('.png', '.jpg', '.jpeg'))])
                label_to_images[class_idx].extend(image_paths)

        # Tạo danh sách ảnh và nhãn xen kẽ
        self.image_paths = []
        self.labels = []
        for label, images in label_to_images.items():
            for img in images:
                self.image_paths.append(img)
                self.labels.append(label)

    def __len__(self):
        return len(self.labels)  # Tổng số lượng ảnh

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Mở ảnh và áp dụng transform nếu có
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label)

        return image, label

    def show_image(self, idx):
        image, label = self.__getitem__(idx)
        keys = [k for k, v in self.labels_dict.items() if v == label]
        
        # Hiển thị ảnh
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Label: {keys[0]}")
        plt.show()


In [61]:
from torch.utils.data import Dataset, Subset

class CustomSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = Subset(dataset, indices)
        self.indices = indices
        self.labels_dict = dataset.labels_dict  # Giữ nguyên labels_dict từ dataset gốc
        self.dataset_cls = dataset  # Duy trì tham chiếu tới dataset gốc

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        return self.dataset[idx]

    def get_class_counts(self):
        """Tính toán số lượng ảnh trong mỗi lớp."""
        class_counts = {class_name: 0 for class_name in self.labels_dict.keys()}
        for idx in self.indices:
            label = self.dataset_cls.labels[idx]
            class_name = [k for k, v in self.labels_dict.items() if v == label][0]
            class_counts[class_name] += 1
        return class_counts

    def show_image(self, idx):
        """Hiển thị ảnh."""
        actual_idx = self.indices[idx]
        self.dataset_cls.show_image(actual_idx)

In [62]:
import torchvision.transforms as transforms 
# Định nghĩa các bước tiền xử lý 
transform = transforms.Compose([ transforms.Resize((128, 128)),   
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [63]:
train_dataset = DERMNET_DATASET('/kaggle/input/dermnet', train=True, transform=transform)

In [64]:
from torchvision.models import convnext_tiny, densenet121, mobilenet_v2, resnet50
import timm

In [65]:
class ConvNeXtTinyModel(nn.Module):
    def __init__(self, num_classes=23, pretrained=True, frozen=False):
        super(ConvNeXtTinyModel, self).__init__()
        
        # Load pre-trained ConvNeXt Tiny model
        self.model = convnext_tiny(pretrained=pretrained)
        
        # Optionally freeze backbone layers
        if frozen:
            for param in self.model.features.parameters():
                param.requires_grad = False
        
        # Replace the classifier layer (fully connected layer) to match the number of classes
        in_features = self.model.classifier[2].in_features
        self.model.classifier[2] = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        return self.model(x)


class DenseNet121(nn.Module):
    def __init__(self, num_classes=10, pretrained=True, frozen=False):
        super(DenseNet121, self).__init__()
        self.model = densenet121(pretrained=pretrained)
        
        if frozen:
            for param in self.model.features.parameters():  # Freeze chỉ các lớp feature extractor
                param.requires_grad = False
        
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        # Truyền input qua mô hình DenseNet121
        return self.model(x)

class MobileNetV2(nn.Module):
    def __init__(self, num_classes=10, frozen=False):
        super(MobileNetV2, self).__init__()
        self.model = mobilenet_v2(pretrained=True)
        if frozen:
            for param in self.model.parameters():
                param.requires_grad = False
        num_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.model(x)

class EfficientNetV2(nn.Module):
    def __init__(self, num_classes=10, frozen=False):
        super(EfficientNetV2, self).__init__()
        self.model = timm.create_model('efficientnetv2_rw_t', pretrained=True)
        if frozen:
            for param in self.model.parameters():
                param.requires_grad = False
            for param in self.model.classifier.parameters():
                param.requires_grad = True
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.model(x)

class ResNet50(nn.Module):
    def __init__(self, num_classes=10, frozen=False):
        super(ResNet50, self).__init__()
        self.model = resnet50(pretrained=True)
        if frozen:
            for param in self.model.parameters():
                param.requires_grad = False
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.model(x)

class ViTClassifier128(nn.Module):
    def __init__(self, num_classes=23, pretrained=True, img_size=128, frozen=False):
        super(ViTClassifier128, self).__init__()
        
        # Tải mô hình ViT với kích thước ảnh 128x128
        self.vit = timm.create_model(
            'vit_base_patch16_224',
            pretrained=pretrained,
            img_size=img_size
        )
        
        # Freeze backbone nếu cần
        if frozen:
            for param in self.vit.parameters():
                param.requires_grad = False

        # Lấy số lượng đầu ra từ lớp head
        in_features = self.vit.head.in_features
        
        # Thay thế lớp head bằng lớp phân loại tùy chỉnh
        self.vit.head = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.vit(x)

In [66]:
convnext_frozen_folder = "/kaggle/input/kfold_models/other/default/1/stacking/convnext_frozen"
convnext_unfrozen_folder ="/kaggle/input/kfold_models/other/default/1/stacking/convnext_unfrozen"  
densenet_frozen_folder = "/kaggle/input/kfold_models/other/default/1/stacking/densenet_frozen"
densenet_unfrozen_folder ="/kaggle/input/kfold_models/other/default/1/stacking/densenet_unfrozen" 
mobilenet_frozen_folder = "/kaggle/input/kfold_models/other/default/1/stacking/mobilenet_frozen"
mobilenet_unfrozen_folder ="/kaggle/input/kfold_models/other/default/1/stacking/mobilenet_unfrozen" 
efficientnet_frozen_folder ="/kaggle/input/kfold_models/other/default/1/stacking/efficientnet_frozen" 
efficientnet_unfrozen_folder ="/kaggle/input/kfold_models/other/default/1/stacking/efficientnet_unfrozen" 
resnet50_frozen_folder = "/kaggle/input/kfold_models/other/default/1/stacking/resnet50_frozen"
resnet50_unfrozen_folder ="/kaggle/input/kfold_models/other/default/1/stacking/resnet50_unfrozen" 
vit_frozen_folder = "/kaggle/input/kfold_models/other/default/1/stacking/vit_frozen"
vit_unfrozen_folder = "/kaggle/input/vit_unfrozen/other/default/1/vit_unfrozen"

base_folders = [convnext_frozen_folder,convnext_unfrozen_folder,densenet_frozen_folder,
                densenet_unfrozen_folder,mobilenet_frozen_folder,mobilenet_unfrozen_folder,
                efficientnet_frozen_folder,efficientnet_unfrozen_folder,resnet50_frozen_folder,
                resnet50_unfrozen_folder,vit_frozen_folder,vit_unfrozen_folder]

base_models = [
    ConvNeXtTinyModel, ConvNeXtTinyModel, DenseNet121, DenseNet121,
    MobileNetV2, MobileNetV2, EfficientNetV2,
    EfficientNetV2, ResNet50, ResNet50, ViTClassifier128, ViTClassifier128
]

base_models_string = [
    'convnext_frozen', 'convnext_unfrozen', 'densenet_frozen', 'densenet_unfrozen',
    'mobilenet_frozen', 'mobilenet_unfrozen', 'efficientnet_frozen',
    'efficientnet_unfrozen', 'resnet50_frozen', 'resnet50_unfrozen', 'vit_frozen','vit_unfrozen'
]

file_names = ['best_model_fold_1.pth', 'best_model_fold_2.pth', 'best_model_fold_3.pth', 'best_model_fold_4.pth', 'best_model_fold_5.pth']

# Define initialization parameters for each model
model_params = {
    "convnext_frozen": {"num_classes": 23, "pretrained": True, "frozen": True},
    "convnext_unfrozen": {"num_classes": 23, "pretrained": True, "frozen": False},
    "densenet_frozen": {"num_classes": 23, "pretrained": True, "frozen": True},
    "densenet_unfrozen": {"num_classes": 23, "pretrained": True, "frozen": False},
    "mobilenet_frozen": {"num_classes": 23, "frozen": True},
    "mobilenet_unfrozen": {"num_classes": 23, "frozen": False},
    "efficientnet_frozen": {"num_classes": 23, "frozen": True},
    "efficientnet_unfrozen": {"num_classes": 23, "frozen": False},
    "resnet50_frozen": {"num_classes": 23, "frozen": True},
    "resnet50_unfrozen": {"num_classes": 23, "frozen": False},
    "vit_frozen": {"num_classes": 23, "pretrained": True, "img_size": 128, "frozen": True},
    "vit_unfrozen": {"num_classes": 23, "pretrained": True, "img_size": 128, "frozen": False},
}


def precompute_model_outputs(base_models, loader, device):
    """
    Lưu trước đầu ra của từng mô hình cơ sở.
    """
    model_outputs = {}
    labels_list = []

    for images, labels in tqdm(loader, desc="Extracting Outputs"):
        images = images.to(device)
        batch_outputs = []

        for i, model in enumerate(base_models):
            model.eval()
            with torch.no_grad():
                output = model(images).cpu().numpy()
                if i not in model_outputs:
                    model_outputs[i] = []
                model_outputs[i].append(output)
        
        labels_list.append(labels.cpu().numpy())

    # Gộp toàn bộ batch lại
    for i in model_outputs:
        model_outputs[i] = np.vstack(model_outputs[i])
    labels = np.hstack(labels_list)

    return model_outputs, labels

In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [68]:
batch_size=128

In [69]:
dataset_indices = np.arange(len(train_dataset))
labels = np.array(train_dataset.labels)  # Tạo bản sao an toàn

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset

# Function to load model state dict
def load_model(model, state_dict_path, device):
    model.load_state_dict(torch.load(state_dict_path, map_location=device))
    model.eval()
    return model
    
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=SEED)

outputs_dict = {}

# Duyệt qua các thư mục của base model
for folder_idx, folder in enumerate(base_folders):
    model_name = base_models_string[folder_idx]
    model_class = base_models[folder_idx]
    params = model_params[model_name]  # Get parameters for the current model

    # Initialize model with specific parameters
    model = model_class(**params).to(device)
    print(f"Processing model: {model_name}")

    for fold_idx, file_name in enumerate(file_names):
        # Lấy danh sách các chỉ số của dataset
        dataset_indices = np.arange(len(train_dataset))
        labels = train_dataset.labels  # Giả định nhãn đã được gán trong dataset
        file_path = os.path.join(folder, file_name)
        print(f"Loading weights from {file_path}")
        
        # Load model weights
        model = load_model(model, file_path, device)

        # Dự đoán trên tập validation của fold hiện tại
        val_indices = list(skf.split(dataset_indices, labels))[fold_idx][1]  # Get validation indices for this fold
        val_subset = Subset(train_dataset, val_indices)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        fold_outputs = []
        fold_labels = []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Predicting fold {fold_idx + 1}"):
                images = images.to(device)
                outputs = model(images)

                fold_outputs.append(outputs.cpu().numpy())
                fold_labels.append(labels.numpy())

        # Kết hợp các batch lại thành một array duy nhất
        fold_outputs = np.vstack(fold_outputs)
        fold_labels = np.hstack(fold_labels)

        # Lưu kết quả vào dictionary
        key = f"{model_name}_fold_{fold_idx + 1}"
        outputs_dict[key] = {
            "predictions": fold_outputs,
            "labels": fold_labels
        }

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_23/1403438812.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for 

Processing model: convnext_frozen
Loading weights from /kaggle/input/kfold_models/other/default/1/stacking/convnext_frozen/best_model_fold_1.pth


Predicting fold 1: 100%|██████████| 25/25 [00:19<00:00,  1.26it/s]
/tmp/ipykernel_23/1403438812.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(to

Loading weights from /kaggle/input/kfold_models/other/default/1/stacking/convnext_frozen/best_model_fold_2.pth


Predicting fold 2: 100%|██████████| 25/25 [00:20<00:00,  1.21it/s]
/tmp/ipykernel_23/1403438812.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(to

...

Predicting fold 5: 100%|██████████| 25/25 [00:20<00:00,  1.22it/s]
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_23/1403438812.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.

Processing model: densenet_frozen
Loading weights from /kaggle/input/kfold_models/other/default/1/stacking/densenet_frozen/best_model_fold_1.pth


Predicting fold 1: 100%|██████████| 25/25 [00:19<00:00,  1.25it/s]
/tmp/ipykernel_23/1403438812.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(to

Loading weights from /kaggle/input/kfold_models/other/default/1/stacking/densenet_frozen/best_model_fold_2.pth


Predicting fold 2:  68%|██████▊   | 17/25 [00:14<00:06,  1.21it/s]

In [ ]:
for key, value in outputs_dict.items():
    predictions = value["predictions"]
    labels = value["labels"]
    print(f"{key}: predictions shape = {predictions.shape}, labels shape = {labels.shape}")


In [ ]:
# Lưu kết quả đầu ra vào file numpy
np.savez("model_outputs.npz", **outputs_dict)
print("All model outputs saved to model_outputs.npz")

In [ ]:
from collections import defaultdict
import numpy as np

# Tạo dictionary để nhóm theo model
model_outputs = defaultdict(lambda: {"predictions": [], "labels": []})

# Nhóm dữ liệu theo model_name
for key, value in outputs_dict.items():
    model_name = key.split("_fold_")[0]  # Tách model_name từ key
    model_outputs[model_name]["predictions"].append(value["predictions"])
    model_outputs[model_name]["labels"].append(value["labels"])

# Nối predictions và labels theo từng model
for model_name, data in model_outputs.items():
    # Nối predictions và labels của từng model
    model_outputs[model_name]["predictions"] = np.vstack(data["predictions"])
    model_outputs[model_name]["labels"] = np.hstack(data["labels"])
    print(f"Model: {model_name}")
    print(f"Combined predictions shape: {model_outputs[model_name]['predictions'].shape}")
    print(f"Combined labels shape: {model_outputs[model_name]['labels'].shape}")


In [ ]:
import json

# Chuyển đổi numpy arrays thành list
model_outputs_serializable = {
    model_name: {
        "predictions": data["predictions"].tolist(),
        "labels": data["labels"].tolist(),
    }
    for model_name, data in model_outputs.items()
}

# Lưu model_outputs vào file .json
with open("model_outputs.json", "w") as f:
    json.dump(model_outputs_serializable, f)
print("Đã lưu model_outputs vào file model_outputs.json")
